# Transferul Stilului Artistic


## Introducere

În acest laborator vom antrena o rețea convoluțională care primește ca input două imagini (o imagine conținut și o imagine pentru stil) și creează o imagine eterogenă, ce conține contururile imaginii-conținut și culorile și textura imaginii-stil. Acest lucru se realizează prin definirea unor funcții de loss ce pot fi optimizate.

Funcția de loss pentru imaginea-conținut încearcă să minimizeze diferența dintre descriptorii care se activează în imaginea-conținut și cei care se activează în imaginea rezultat, la nivelul unuia sau al mai multor layere. Asta are drept rezultat păstrarea contururilor din imaginea conținut în rezultat. 

Funcția de loss pentru imaginea-stil este puțin mai complicată pentru că încearcă să minimizeze diferența dintre așa-numitele matrice [*Gram*](https://en.wikipedia.org/wiki/Gramian_matrix) pentru imaginea-stil și imaginea finală (la nivelul unuia sau al mai multor layere). Matricea Gram măsoară care descriptori sunt activați simultan într-un anumit layer. Alterând imaginea mixată astfel încât să imite tiparele de activare ale imaginii-stil are drept rezultat transferul culorii și texturii către aceasta.

Vom folosi Tensorflow pentru a calcula gradienții acestor funcții de loss. Acest gradient este ulterior folosit pentru a actualiza imaginea mixată, într-un proces iterativ, până când suntem mulțumiți cu rezultatul obținut.

![alt text](http://dev.wode.ai/repo/TensorFlow-Tutorials-HvassLabs/images/15_style_transfer_flowchart.svg)



In [0]:
from google.colab import files
uploaded = files.upload()
print("OK")

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import PIL.Image
from IPython.display import Image, display

tf.__version__

## Modelul VGG-16

Descarcă modelul VGG-16 dacă nu a fost deja descărcat.

In [0]:
import vgg16

# vgg16.data_dir = 'vgg16/'
vgg16.maybe_download()

## Funcții ajutătoare pentru manipularea imaginilor

Această funcție încarcă o imagine; imaginea poate fi redimensionată astfel încât cea mai mare latură să fie `max_size`.

In [0]:
def load_image(filename, max_size=None):
    image = PIL.Image.open(filename)

    if max_size is not None:
        # Calculează factorul de scalare necesat pentru
        # a asigura înălțimea și lățimea maxime, păstrând,
        # în același timp, proporțiile dintre acestea.
        factor = max_size / np.max(image.size)
    
        # Redimensionează imaginea
        size = np.array(image.size) * factor
        size = size.astype(int)
        
        image = image.resize(size, PIL.Image.LANCZOS)

    return np.float32(image)

Salvează imaginea.

In [0]:
def save_image(image, filename):
    # Asigură că valorile pixelilor sunt în [0, 255]
    image = np.clip(image, 0.0, 255.0)
    
    # Convertește în bytes
    image = image.astype(np.uint8)
    
    # Scrie imaginea
    with open(filename, 'wb') as file:
        PIL.Image.fromarray(image).save(file, 'jpeg')

Plotează imaginea.

In [0]:
def plot_image_big(image):
    # Asigură că valorile pixelilor sunt în [0, 255]
    image = np.clip(image, 0.0, 255.0)

    # Convertește în bytes
    image = image.astype(np.uint8)

    # Afișează imaginea
    display(PIL.Image.fromarray(image))

Afișează imaginile-conținut, -mixată și -stil.

In [0]:
def plot_images(content_image, style_image, mixed_image):
    fig, axes = plt.subplots(1, 3, figsize=(10, 10))

    fig.subplots_adjust(hspace=0.1, wspace=0.1)

    smooth = True
    if smooth:
        interpolation = 'sinc'
    else:
        interpolation = 'nearest'

    # Afișează imaginea-conținut
    ax = axes.flat[0]
    ax.imshow(content_image / 255.0, interpolation=interpolation)
    ax.set_xlabel("Content")

    # Afișează imaginea-mix
    ax = axes.flat[1]
    ax.imshow(mixed_image / 255.0, interpolation=interpolation)
    ax.set_xlabel("Mixed")

    # Afișează imaginea-stil
    ax = axes.flat[2]
    ax.imshow(style_image / 255.0, interpolation=interpolation)
    ax.set_xlabel("Style")

    for ax in axes.flat:
        ax.set_xticks([])
        ax.set_yticks([])
    
    plt.show()

## Funcțiile de loss

Metode ajutătoare pentru definirea funcțiilor de loss.

In [0]:
# Calculează MSE între 2 tensori

def mean_squared_error(a, b):
    return tf.reduce_mean(tf.square(a - b))

Creează funcția de loss pentru imaginea-conținut: MSE între activările de pe un anumit layer al imaginii-conținut și al imaginii-mix. Când această funcție este minimizată, imaginea-mix va avea activările din layerul respectiv asemănătoare cu activările imaginii-conținut. În funcție de layerul selectat, aceasta ar trebui să transfere contururile din imaginea-conținut în imaginea-mix.

In [0]:
def create_content_loss(session, model, content_image, layer_ids):
    """
    Creează funcția de loss a imaginii-conținut.
    
    Parametri:
    session: sesiune Tensorflow pentru rularea grafului modelului.
    model: modelul (instanță a clasei VGG)
    content_image: array numpy reprezentând imaginea-conținut
    layer_ids: Listă de id-uri de layere
    """
    
    feed_dict = model.create_feed_dict(image=content_image)

    # Obține referințele către tensorii layerelor.
    layers = model.get_layer_tensors(layer_ids)

    # Calculează rezultatele tensorilor respectivi
    values = session.run(layers, feed_dict=feed_dict)

    with model.graph.as_default():
        # Listă vidă pentru funcțiile de loss
        layer_losses = []
    
        # Pentru fiecare layer
        for value, layer in zip(values, layers):
            value_const = tf.constant(value)

            # Calculează MSE
            loss = mean_squared_error(layer, value_const)

            # Adaugă la lista de loss-uri
            layer_losses.append(loss)

        # Calculează media loss-urilor
        total_loss = tf.reduce_mean(layer_losses)
        
    return total_loss

Vom aplica e metodă similară pentru layerele de stil, doar că acum dorim să măsurăm care descriptori din layerele de stil se activează simultan pentru imaginea-stil, ca apoi să copiem aceste activări în imaginea-mix.

O metodă pentru a realiza asta este să calculăm matricea Gram pentru tensorii layerelor de stil (matricea Gram este o matrice a produsului scalar a vectorilor ce reprezintă activările unui layer).

Dacă o valoare din matricea Gram este apropiată de zero, înseamnă că cei doi descriptori din layerul respectiv nu se activează simultan (și vice-versa, dacă o valoare in matricea Gram este mare, înseamnă că cei doi descriptori din layerul respectiv se activează simultan). Astfel, vom încerca să obținem o imagine-mix care reproduce activările din imaginea-stil.

Aceasta este funcția ajutătoare pentru calcularea matricei Gram a unui layer într-o rețea convoluțională.

In [0]:
def gram_matrix(tensor):
    shape = tensor.get_shape()
    
    # Obține numărul de canale.
    num_channels = int(shape[3])

    matrix = tf.reshape(tensor, shape=[-1, num_channels])
    
    # Calculează matricea Gram ca produs scalar între
    # toate combinațiile de 2 canale din tensor
    gram = tf.matmul(tf.transpose(matrix), matrix)

    return gram                            

Funcția de loss pentru imaginea-stil.

In [0]:
def create_style_loss(session, model, style_image, layer_ids):
    """
    Funcția loss pentru imaginea-stil.
    
    Parametri:
    session: sesiune Tensorflow pentru rularea grafului modelului.
    model: modelul (instanță a clasei VGG)
    style_image: array numpy reprezentând imaginea-stil
    layer_ids: Listă de id-uri de layere
    """

    feed_dict = model.create_feed_dict(image=style_image)

    # Obține referințele către tensorii layerelor.
    layers = model.get_layer_tensors(layer_ids)

    with model.graph.as_default():
        # Operațiile Tensorflow pentru calculul matricelor Gram
        gram_layers = [gram_matrix(layer) for layer in layers]

        # Calculează valorile matricelor Gram
        values = session.run(gram_layers, feed_dict=feed_dict)

        # Listă vidă pentru loss-uri
        layer_losses = []
    
        # Pentru fiecare matrice Gram
        for value, gram_layer in zip(values, gram_layers):
            value_const = tf.constant(value)

            # Calculează MSE
            loss = mean_squared_error(gram_layer, value_const)

            # Adaugă la lista de loss-uri
            layer_losses.append(loss)

        # Calculează media
        total_loss = tf.reduce_mean(layer_losses)
        
    return total_loss

Funcția de loss pentru denoising ([Total Variation Denoising](https://en.wikipedia.org/wiki/Total_variation_denoising)). Aceasta mută imaginea-mix un pixel pe axele *x* și *y*, calculează diferența fața de imaginea originală, pe care o însumează pentru toți pixelii din imagine. Această funcție poate fi folosită pentru a elimina o parte din artefactele din imaginea-mix.

In [0]:
def create_denoise_loss(model):
    loss = tf.reduce_sum(tf.abs(model.input[:,1:,:,:] - model.input[:,:-1,:,:])) + \
           tf.reduce_sum(tf.abs(model.input[:,:,1:,:] - model.input[:,:,:-1,:]))

    return loss

## Algoritmul de Transfer al Stilului

Aplică SGD pentru funcțiile de loss definite anterior. Totodată, normalizează funcțiile de loss, pentru a permite ponderarea loss-urilor de conținut și stil.

In [0]:
def style_transfer(content_image, style_image,
                   content_layer_ids, style_layer_ids,
                   weight_content=1.5, weight_style=10.0,
                   weight_denoise=0.3,
                   num_iterations=120, step_size=10.0):
    """
    Aplică SGD pentru a minimica loss-urile layerelor de conținut
    și de stil; asta ar trebui să rezulte într-o imagine care
    păstrează contururile din imaginea-conținut, respectiv culoarea
    și textura imaginii-stil.
    
    Parametri:
    content_image: Imaginea-conținut
    style_image: Imaginea-stil
    content_layer_ids: Lista id-urilor layere-lor de conținut
    style_layer_ids: Lista id-urilor layere-lor de stil
    weight_content: Ponderea loss-ului de conținut
    weight_style: Ponderea loss-ului de stil
    weight_denoise: Ponderea loss-ului de denoise
    num_iterations: Numărul de iterații
    step_size: Dimensiunea unui pas al gradientului în fiecare iterație
    """

    # Creează o instanță a modelului VGG16
    model = vgg16.VGG16()

    # Creează o sesiune Tensorflow
    session = tf.InteractiveSession(graph=model.graph)

    # Printează denumirea layar-elor-conținut
    print("Content layers:")
    print(model.get_layer_names(content_layer_ids))
    print()

    # Printează denumirea layer-elor-stil
    print("Style layers:")
    print(model.get_layer_names(style_layer_ids))
    print()

    # Creează loss-ul de conținut
    loss_content = create_content_loss(session=session,
                                       model=model,
                                       content_image=content_image,
                                       layer_ids=content_layer_ids)

    # Creează loss-ul de stil
    loss_style = create_style_loss(session=session,
                                   model=model,
                                   style_image=style_image,
                                   layer_ids=style_layer_ids)    

    # Creează loss-ul de denoise
    loss_denoise = create_denoise_loss(model)

    # Variabile pentru ponderile loss-urilor
    adj_content = tf.Variable(1e-10, name='adj_content')
    adj_style = tf.Variable(1e-10, name='adj_style')
    adj_denoise = tf.Variable(1e-10, name='adj_denoise')

    session.run([adj_content.initializer,
                 adj_style.initializer,
                 adj_denoise.initializer])

    # Operații Tensorflow pentru actualizarea ponderilor
    update_adj_content = adj_content.assign(1.0 / (loss_content + 1e-10))
    update_adj_style = adj_style.assign(1.0 / (loss_style + 1e-10))
    update_adj_denoise = adj_denoise.assign(1.0 / (loss_denoise + 1e-10))

    # Media ponderată a loss-urilor (pe aceasta o vom minimiza)
    loss_combined = weight_content * adj_content * loss_content + \
                    weight_style * adj_style * loss_style + \
                    weight_denoise * adj_denoise * loss_denoise

    gradient = tf.gradients(loss_combined, model.input)

    # Lista tensorilor pe care-i vom actualiza
    run_list = [gradient, update_adj_content, update_adj_style, \
                update_adj_denoise]

    # Inițializează random imaginea-mix
    mixed_image = np.random.rand(*content_image.shape) + 128

    for i in range(num_iterations):
        feed_dict = model.create_feed_dict(image=mixed_image)

        # Calculează valoarea gradientului
        grad, adj_content_val, adj_style_val, adj_denoise_val \
        = session.run(run_list, feed_dict=feed_dict)

        # Reduce dimensionalitatea gradientului
        grad = np.squeeze(grad)

        step_size_scaled = step_size / (np.std(grad) + 1e-8)

        # Actualizează imaginea-mix
        mixed_image -= grad * step_size_scaled

        # Asigură că valorile pixelilor sunt în [0.0, 255.0]
        mixed_image = np.clip(mixed_image, 0.0, 255.0)

        print(". ", end="")

        # Afișează status la fiecare 10 iterații
        if (i % 10 == 0) or (i == num_iterations - 1):
            print()
            print("Iteration:", i)

            # Afișează ponderi
            msg = "Weight Adj. for Content: {0:.2e}, Style: {1:.2e}, Denoise: {2:.2e}"
            print(msg.format(adj_content_val, adj_style_val, adj_denoise_val))

            # Afișează imaginile
            plot_images(content_image=content_image,
                        style_image=style_image,
                        mixed_image=mixed_image)
            
    print()
    print("Final image:")
    plot_image_big(mixed_image)

    session.close()
    
    return mixed_image

## Exemplu

Încarcă imaginea-conținut.

In [0]:
# content_filename = 'images/willy_wonka_old.jpg'
content_filename = 'willy_wonka_new.jpg'
content_image = load_image(content_filename, max_size=None)

Încarcă imaginea-stil.

In [0]:
# style_filename = 'images/style7.jpg'
style_filename = 'style3.jpg'
style_image = load_image(style_filename, max_size=300)

Listă de id-uri de layere pentru imaginea-conținut.

In [0]:
content_layer_ids = [4]

Listă de id-uri de layere pentru imaginea-stil.

In [0]:
# Modelul VGG-16 are 13 layere convoluționale.
# Aceasta selectează toate layerele.
style_layer_ids = list(range(13))

# Puteți selecta și un subset de layere
# style_layer_ids = [1, 2, 3, 4]

Aplică transferul de stil

In [0]:
%%time
img = style_transfer(content_image=content_image,
                     style_image=style_image,
                     content_layer_ids=content_layer_ids,
                     style_layer_ids=style_layer_ids,
                     weight_content=1.5,
                     weight_style=10.0,
                     weight_denoise=0.3,
                     num_iterations=60,
                     step_size=10.0)

## Exerciții

* Încercați să optimizați timp de mai multe iterații (de exemplu 1000, 5000) și cu step-size mai mic. îmbunătățește calitatea?
* Modificați ponderile pentru stil, conținut și denoising.
* Încercați să începeți optimizarea fie de la imaginea-conținut, fie de la imaginea-stil, sau de la o combinație (medie) a acestora. Puteți, de asemenea, să adăugați și puțin noise.
* Încercați să modificați rezoluția imaginilor-conținut și -stil. Puteți folosi argumentul `max_size` al funcției `load_image()` pentru a redimensiona imaginile. Cum afectează rezultatul?
* Încercați să folosiți și alte layere din model.
* Păstrați parametrii constanți pe toată durata optimizării. Cum afectează rezultatul?
* Înlocuiți SGD cu ADAM.
* Folosiți alte modele pre-antrenate.